# GPU

查看显卡信息

In [1]:
!nvidia-smi

Mon Dec  4 21:29:20 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.45                 Driver Version: 531.52       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 L...    On | 00000000:01:00.0  On |                  N/A |
| N/A   38C    P5                6W /  N/A|   1283MiB /  8188MiB |     10%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

查询可用gpu的数量

In [3]:
torch.cuda.device_count()

1

这两个函数允许我们在不存在所需所有GPU的情况下运行代码

In [4]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

查询张量所在的设备

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

存储在GPU上

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

第二个GPU上创建一个随机张量

In [12]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.2945, 0.2031, 0.5300],
        [0.3940, 0.1979, 0.0904]], device='cuda:0')

要计算`X + Y`，我们需要决定在哪里执行这个操作

In [13]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


现在数据在同一个GPU上（`Z`和`Y`都在），我们可以将它们相加

In [14]:
Y + Z

tensor([[1.2945, 1.2031, 1.5300],
        [1.3940, 1.1979, 1.0904]], device='cuda:0')

In [15]:
Z.cuda(0) is Z

True

神经网络与GPU

In [16]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.1272],
        [-0.1272]], device='cuda:0', grad_fn=<AddmmBackward0>)

确认模型参数存储在同一个GPU上

In [17]:
net[0].weight.data.device

device(type='cuda', index=0)

## 练习

In [18]:
import time

startTime1=time.time()
for i in range(100):
    A = torch.ones(500,500)
    B = torch.ones(500,500)
    C = torch.matmul(A,B)
endTime1=time.time()

startTime2=time.time()
for i in range(100):
    A = torch.ones(500,500,device=try_gpu())
    B = torch.ones(500,500,device=try_gpu())
    C = torch.matmul(A,B)
endTime2=time.time()

print('cpu计算总时长:', round((endTime1 - startTime1)*1000, 2),'ms')
print('gpu计算总时长:', round((endTime2 - startTime2)*1000, 2),'ms')

cpu计算总时长: 3463.24 ms
gpu计算总时长: 213.35 ms


In [2]:
import time
import torch

def try_gpu(i=0):  
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

startTime1=time.time()
for i in range(100):
    A = torch.ones(500,500)
    B = torch.ones(500,500)
    C = torch.matmul(A,B)
endTime1=time.time()

startTime2=time.time()
for i in range(100):
    A = torch.ones(500,500,device=try_gpu())
    B = torch.ones(500,500,device=try_gpu())
    C = torch.matmul(A,B)
endTime2=time.time()

print('cpu计算总时长:', round((endTime1 - startTime1)*1000, 2),'ms')
print('gpu计算总时长:', round((endTime2 - startTime2)*1000, 2),'ms')

cpu计算总时长: 3333.83 ms
gpu计算总时长: 464.69 ms
